In [ ]:
import openai
from openai import OpenAI
import time
key = ""
openai.api_key = key

In [ ]:
ASSISTANT_ID = "asst_3rvG2uMMSiIAMdvaljKpj1dw"

# Make sure your API key is set as an environment variable.
client = OpenAI(api_key=key)

In [ ]:
# # 1st : Generate 5 common outputs
# def gen_outputs(order_loc, return_loc, payment_loc, output):
#     if order_life_cycle 

In [ ]:
# 2nd : add dataset and ask question 
def add_data_n_question(Data_loc,Question):
    message_file = client.files.create(
        file=open(Data_loc, "rb"), purpose="assistants"
    )
    thread = client.beta.threads.create(
        messages=[
                {
                    "role": "user",
                    # Update this with the query you want to use.
                    "content": Question,
                    "attachments": [
                                    { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
                                ],
                }
            ]
    )
    run = client.beta.threads.runs.create(thread_id=thread.id, 
                                          assistant_id=ASSISTANT_ID,
                                        #   messages=[
                                        #                 {
                                        #                     "role": "user",
                                        #                     # Update this with the query you want to use.
                                        #                     "content": Question,
                                        #                     "attachments": [
                                        #                                     { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
                                        #                                 ],
                                        #                 }
                                        #             ])
    )
    print(f"👉 Run Created: {run.id}")

    # Wait for run to complete.
    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        print(f"🏃 Run Status: {run.status}")
        time.sleep(1)
    else:
        print(f"🏁 Run Completed!")

    # Get the latest message from the thread.
    message_response = client.beta.threads.messages.list(thread_id=thread.id)
    messages = message_response.data

    # Print the latest message.
    latest_message = messages[0]
    for i in latest_message.content:
        print(i)
        if "ImageFileContentBlock" in str(i):
            # print("yes")
            api_response = client.files.with_raw_response.retrieve_content(latest_message.content[0].image_file.file_id)

            if api_response.status_code == 200:
                content = api_response.content
                with open('image.png', 'wb') as f:
                    f.write(content)
                print('File downloaded successfully.')
        elif "TextContentBlock" in str(i):
            # print("yes1")
            print(f"💬 Response: {i.text.value}")   

In [ ]:
# 3rd : Ask more questions on the same data
def ask_new_question(Data_loc, Question):
    message_file = client.files.create(
        file=open(Data_loc, "rb"), purpose="assistants"
    )
    thread = client.beta.threads.create(
        messages=[
                {
                    "role": "user",
                    # Update this with the query you want to use.
                    "content": Question,
                    "attachments": [
                                    { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
                                ],
                }
            ]
    )
    run = client.beta.threads.runs.create(thread_id=thread.id, 
                                          assistant_id=ASSISTANT_ID,
                                        #   messages=[
                                        #                 {
                                        #                     "role": "user",
                                        #                     # Update this with the query you want to use.
                                        #                     "content": Question,
                                        #                 }
                                        #             ]
                                        )
    print(f"👉 Run Created: {run.id}")

    # Wait for run to complete.
    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        print(f"🏃 Run Status: {run.status}")
        time.sleep(1)
    else:
        print(f"🏁 Run Completed!")

    # Get the latest message from the thread.
    message_response = client.beta.threads.messages.list(thread_id=thread.id)
    messages = message_response.data

    # Print the latest message.
    latest_message = messages[0]
    for i in latest_message.content:
        print(i)
        if "ImageFileContentBlock" in str(i):
            print("yes")
            api_response = client.files.with_raw_response.retrieve_content(latest_message.content[0].image_file.file_id)

            if api_response.status_code == 200:
                content = api_response.content
                with open('image.png', 'wb') as f:
                    f.write(content)
                print('File downloaded successfully.')
        elif "TextContentBlock" in str(i):
            print("yes1")
            print(f"💬 Response: {i.text.value}")  
            # return 

In [ ]:
flag = True
while flag:
    option = int(input("1. Generate 5 common outputs.\n2.Add new Data and question.\n3.Ask new question on same data.\n4.Exit"))
#    thread = client.beta.threads.create()
    # if option == 1:
    #     order_loc = input("Add location of Orders Data")
    #     return_loc = input("Add location of Returns Data")
    #     payment_loc = input("Add location of Payments Data")
    #     generate_visualization = input()
    #     gen_outputs()

    if option == 2:
        data_loc = input("Add location of new data:")
        question = input("Ask your question:")
        if ".txt" in str(data_loc):
            add_data_n_question(data_loc,question)
            pass
        else:
            print("Data must be in txt format.")

    elif option == 3:
        data_loc = input("Read location of existing data:")
        question = input("Ask another question")
        ask_new_question(Data_loc = data_loc, Question= question)
        #ask_new_question(combined_input) # type: ignore

    elif option == 4:
        print("Thank you!!")
        flag = False
    else:
        print("Choose the correcct option!!")